### 引入相关模块

In [46]:
import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms

### 定义超参数

In [64]:
EPOCHS = 10                 # 迭代次数
BATCH_SIZE = 64             # batch_size
LEARNING_RATE = 0.001       # 学习率
ACC_FREQUENCE = 300         # 多少个batch打印一次输出
SAVE_PATH = './output/'
# 训练设备
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### 搭建模型

In [54]:
class DemoModel(torch.nn.Module):
    def __init__(self, num_classes, input_channel=3,):
        super(DemoModel, self).__init__()
        self.conv1 = torch.nn.Sequential(
            torch.nn.Conv2d( input_channel, 10, kernel_size=3, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2),
        )
        self.conv2 = torch.nn.Sequential(
                torch.nn.Conv2d( 10, 20, kernel_size=3, padding=1),
                torch.nn.ReLU(),
                torch.nn.MaxPool2d(kernel_size=2),
        )
        self.fc = torch.nn.Sequential(
            torch.nn.Linear(1280, 50),
            torch.nn.Linear(50,num_classes)
        )

    def forward(self, x):
        out = self.conv1(x)
        out = self.conv2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out


### 数据预处理

In [49]:
data_hub = '../data_hub/'
data_transform = {
    "train": transforms.Compose([
            # transforms.RandomResizedCrop(224),
            # transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            # transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
            transforms.Normalize((0.1307,), (0.3081,))
        ]),
    "val": transforms.Compose([
            # transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
            # transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])}

train_dataset = datasets.CIFAR10(root=data_hub, train=True, download=True, transform=data_transform["train"])
validate_dataset = datasets.CIFAR10(root=data_hub, train=False, download=True, transform=data_transform["val"])

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
validate_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False)


Files already downloaded and verified
Files already downloaded and verified


### 查看训练数据

In [50]:

for item in train_loader:
    x,y = item
    print(x.shape, y.shape) 
    break

torch.Size([64, 3, 32, 32]) torch.Size([64])


### 定义更新规则

In [51]:
def useBackward(model, lr):
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr = lr)
    return criterion, optimizer

### 开始训练

In [62]:
def train(model, epochs):
    running_loss = 0.0
    running_total = 0
    running_correct = 0
    # 使用更新方法
    criterion, optimizer = useBackward(model, LEARNING_RATE)
    for epoch in range(epochs):
        for batch_idx, data in enumerate(train_loader, 0):
            inputs, target = data
            inputs, target = inputs.to(DEVICE), target.to(DEVICE)
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()

            # 把运行中的loss累加起来
            running_loss += loss.item()
            # 计算此时的准去略acc
            _, predicted = torch.max(outputs.data, dim=1)
            running_total += inputs.shape[0]
            running_correct += (predicted == target).sum().item()
            # 训练进度
            rate = ( batch_idx + 1) / len(train_loader)
            a = "*" * int( rate * 50)
            b = "." * int((1-rate) * 50)
            print("\rtrain loss: {:^3.0f}%[{}->{}]{:.3f}".format(int(rate * 100), a, b, loss), end="")
        # 每个epoch输出一次结果
        # if batch_idx % ACC_FREQUENCE == (ACC_FREQUENCE-1):  # 没FRE_QUEncE次出一个平均损失,和准确率
        print()
        print('epoch[%d]: loss: %.3f , acc: %.2f %%'
                % (epoch + 1,  running_loss / len(train_loader), 100 * running_correct / running_total))
        running_loss = 0.0  # 这小批的loss清零
        running_total = 0
        running_correct = 0  # 这小批的acc清零
model = DemoModel(10).to(DEVICE)
train(model, 3)

train loss: 100%[**************************************************->]1.374
epoch[ 1 ]: loss: 1.516 , acc: 45.86 %
train loss: 100%[**************************************************->]0.854
epoch[ 2 ]: loss: 1.220 , acc: 57.23 %
train loss: 100%[**************************************************->]0.912
epoch[ 3 ]: loss: 1.097 , acc: 61.89 %


### 模型保存

In [65]:
# 训练完了保存模型
def saveModel(model, file_name='model.pth'):
    with open(SAVE_PATH+file_name, 'wb') as f:
        torch.save(model.state_dict(), f)
saveModel(model)

### 数据可视化

### 模型评估